### Test Gurobi solver

In [12]:
!pwd

/home/ivy/Cornell_PB_NE_Hub/references/Optimization_model_demo


In [24]:
!activate pyomoenv

/bin/sh: 1: activate: not found


In [21]:
help('modules')


Please wait a moment while I gather a list of all available modules...

Crypto              brain_mechanize     jupyter             requests
Cython              brain_multiprocessing jupyter_client      resource
IPython             brain_namedtuple_enum jupyter_console     rlcompleter
Model_demo          brain_nose          jupyter_core        rmagic
OpenSSL             brain_numpy         jupyterhub          rope
PIL                 brain_pkg_resources jupyterlab          ruamel_yaml
PyQt5               brain_pytest        jupyterlab_server   runpy
__future__          brain_qt            keras_applications  sched
_abc                brain_random        keras_preprocessing scipy
_ast                brain_re            keyring             scripts
_asyncio            brain_six           keyword             seaborn
_bisect             brain_ssl           kiwisolver          secrets
_blake2             brain_subprocess    lazy_object_proxy   secretstorage
_bootlocale         brain_threadi

In [13]:
import pyomo

In [11]:
!cd /home/ivy/Cornell_PB_NE_Hub/references/Optimization_model_demo && pyomo solve Model_demo.py Data_demo.dat --solver=gurobi --summary --stream-solver


/bin/sh: 1: pyomo: not found


In [17]:
!conda list

# packages in environment at /home/ubuntu/anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
alabaster                 0.7.12                   py37_0  
alembic                   1.0.2                      py_0    conda-forge
anaconda                  2018.12                  py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.6                    py37_0  
anaconda-project          0.8.2                    py37_0  
asn1crypto                0.24.0                   py37_0  
astroid                   2.1.0                    py37_0  
astropy                   3.1              py37h7b6447c_0  
async_generator           1.10                       py_0    conda-forge
atomicwrites              1.2.1                    py37_0  
attrs                     18.2.0           py37h28b3542_0  
babel                     2.6.0                    py37_0  
backcall    

------

### Below is the Original Demo: run on gecor notebook with glpk solver

### Method 1: Apply solver directly

In [1]:
!cd /home/fusion/work/Optimization_model_demo && pyomo solve Model_demo.py Data_demo.dat --solver=glpk --summary --stream-solver


[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.04] Applying solver
GLPSOL: GLPK LP/MIP Solver, v4.60
Parameter(s) specified in the command line:
 --write /home/fusion/work/Optimization_model_demo/tmpgapa5s2f.glpk.raw --wglp
 /home/fusion/work/Optimization_model_demo/tmp3g_2wv9o.glpk.glp --cpxlp /home/fusion/work/Optimization_model_demo/tmpjf2krqac.pyomo.lp
Reading problem data from '/home/fusion/work/Optimization_model_demo/tmpjf2krqac.pyomo.lp'...
/home/fusion/work/Optimization_model_demo/tmpjf2krqac.pyomo.lp:122: warning: lower bound of variable 'x16' redefined
/home/fusion/work/Optimization_model_demo/tmpjf2krqac.pyomo.lp:122: warning: upper bound of variable 'x16' redefined
9 rows, 19 columns, 34 non-zeros
18 integer variables, 3 of which are binary
125 lines were read
Writing problem data to '/home/fusion/work/Optimization_model_demo/tmp3g_2wv9o.glpk.glp'...
108 lines were written
GLPK Integer Optimizer, v4.

### Method 2: Create model instance

In [2]:
import pandas as pd
import pyomo.opt

from pyomo.environ import *

In [3]:
import Model_demo

In [4]:
opt = pyomo.opt.SolverFactory("glpk")
#create model instance
instance = Model_demo.model.create_instance('/home/fusion/work/Optimization_model_demo/Data_demo.dat')
#apply solver
opt.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5610.0, 'Upper bound': 5610.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 19, 'Number of nonzeros': 34, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.009116888046264648}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [5]:
#obtain results
def get_flow(instance):
    
    origin=[]
    destination=[]
    volume=[]
    
    for v in instance.component_objects(Var, active=True):
        varobject = getattr(instance, str(v))
        if(varobject.name=='x'):
            for index in varobject:
                pkg=varobject[index].value
#                 if pkg > 0:
                origin.append(str(index[0]))
                destination.append(str(index[1]))
                volume.append(round(pkg))
    
    flow_matrix = pd.DataFrame({'Facility':origin, 'Client':destination, 'Packages':volume})
    return flow_matrix

In [6]:
df_flow = get_flow(instance)
df_flow

,Client,Facility,Packages
0,b,4,0
1,b,2,270
2,a,3,0
3,a,1,0
4,a,5,0
5,c,3,100
6,c,2,0
7,c,5,180
8,a,4,0
9,a,2,0


In [7]:
df_flow.to_csv('flow_matrix.csv')